# 전이학습을 이용한 히토미 망가 분류

https://tutorials.pytorch.kr/beginner/transfer_learning_tutorial.html의 예제를 이용합니다.

작품은 이곳에서 받을수있습니다.
[1536576](https://beta.doujinshiman.ga/download/index.html?index=1536576)
[1570712](https://beta.doujinshiman.ga/download/index.html?index=1570712)
[1613730](https://beta.doujinshiman.ga/download/index.html?index=1613730)

폴더 구조는 다음과 같습니다.

```sh
data/
├─ train/
│  ├─ 1536576/
│  ├─ 1570712/
│  ├─ 1613730/
├─ val/
│  ├─ 1570712/
│  ├─ 1613730/
│  ├─ 1536576/
```



In [9]:
# 망가의 이미지만으로는 학습할 데이터셋이 터무니없이 부족합니다.
# keras로 이미지의 변형을 통해 학습할 데이터를 늘려봅시다.

# https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html


import asyncio
from keras.preprocessing.image import (
    ImageDataGenerator,
    array_to_img,
    img_to_array,
    load_img,
)
import os
import copy
import contextvars
import functools

async def to_thread(func, *args, **kwargs):
    """Asynchronously run function *func* in a separate thread.

    Any *args and **kwargs supplied for this function are directly passed
    to *func*. Also, the current :class:`contextvars.Context` is propogated,
    allowing context variables from the main thread to be accessed in the
    separate thread.

    Return a coroutine that can be awaited to get the eventual result of *func*.
    """
    loop = asyncio.get_running_loop()
    ctx = contextvars.copy_context()
    func_call = functools.partial(ctx.run, func, *args, **kwargs)
    return await loop.run_in_executor(None, func_call)

#옵션넣기
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest",
)

# 학습할 데이터 (검증할 데이터로 학습할거기때문에 val 데이터 사용)
MAIN_DIR = "./data/val/"

# 필요한것들 미리선언
label_dict = {}
img_full_dir = []

# 폴더 구조 분석
for folder in os.listdir(MAIN_DIR):
    sub = os.walk(MAIN_DIR + folder)
    for sub_dir_info in sub:
        for img in sub_dir_info[2]:
            img_full_dir.append(sub_dir_info[0] + "/" + img)
        label_dict[folder] = copy.deepcopy(img_full_dir)
        img_full_dir.clear()


# 단일로 하면 너무 느려서 to_thread를 이용해 비동기로 작업해줄거임
async def main():
    os.mkdir("gen")
    # 따로 실행할것들 함수 나눠줌
    def main(k, v):
        os.mkdir("gen/" + k)
        # TODO: 이 부분도 나눠주면 더빠를거같음.
        for img_dir in v:
            img = load_img(img_dir)
            x = img_to_array(img)
            x = x.reshape((1,) + x.shape)
            i = 0
            for batch in datagen.flow(
                x,
                batch_size=1,
                save_to_dir="gen/" + k,
                save_prefix=k,
                save_format="jpeg",
            ):
                i += 1
                if i > 20:
                    break

    # 모일 함수
    funcs = []
    # 라벨이랑 경로있는 dict
    for k, v in label_dict.items():
        # 함수 넣어주기
        funcs.append(to_thread(main, k, v))

    # 동시에 실행
    await asyncio.gather(*funcs)


# https://stackoverflow.com/questions/64074295/is-there-an-equivalence-of-await-in-google-colab
import nest_asyncio
nest_asyncio.apply()

import asyncio
await_func = lambda x: asyncio.get_event_loop().run_until_complete(x)
await_func(main())
print("done")


done



데이터셋이 어마어마 하게 불었습니다.

![bbungtuigi](./img/bbungtuigi.png)

이 데이터들로 학습합시다.

gen폴더를 data폴더로 옮긴후 train으로 이름을 바꿔줍시다.

In [24]:
# 필요한 모듈을 임포트해줍니다.
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

In [28]:
# 이미지 라벨링등 기타작업
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = './data/'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=2)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
# 일부 이미지 시각화
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # 갱신이 될 때까지 잠시 기다립니다.


# 학습 데이터의 배치를 얻습니다.
inputs, classes = next(iter(dataloaders['train']))

# 배치로부터 격자 형태의 이미지를 만듭니다.
out = torchvision.utils.make_grid(inputs)

# R-18이미지가 포함되어있으므로 이곳에서는 표시하지않습니다.
imshow(out, title=[class_names[x] for x in classes])

In [30]:
# 모델을 학습하기 위한 일반 함수
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # 각 에폭(epoch)은 학습 단계와 검증 단계를 갖습니다.
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # 모델을 학습 모드로 설정
            else:
                model.eval()   # 모델을 평가 모드로 설정

            running_loss = 0.0
            running_corrects = 0

            # 데이터를 반복
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # 매개변수 경사도를 0으로 설정
                optimizer.zero_grad()

                # 순전파
                # 학습 시에만 연산 기록을 추적
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # 학습 단계인 경우 역전파 + 최적화
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # 통계
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # 모델을 깊은 복사(deep copy)함
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # 가장 나은 모델 가중치를 불러옴
    model.load_state_dict(best_model_wts)
    return model

In [31]:
# 모델 예측값 시각화하기
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [32]:
# 합성곱 신경망 미세조정
model_ft = models.resnet18(pretrained=True) # 미리학습된 라스넷 모델 사용
num_ftrs = model_ft.fc.in_features
# 여기서 각 출력 샘플의 크기는 3으로 설정합니다.
# 라벨이 3개이기떄문입니다.
# 또는, nn.Linear(num_ftrs, len (class_names))로 일반화할 수 있습니다.
model_ft.fc = nn.Linear(num_ftrs, 3)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# 모든 매개변수들이 최적화되었는지 관찰
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# 7 에폭마다 0.1씩 학습률 감소
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [33]:
# 학습 시작
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=25)

Epoch 0/24
----------
train Loss: 0.9060 Acc: 0.6125
val Loss: 0.1317 Acc: 0.9630

Epoch 1/24
----------
train Loss: 0.7810 Acc: 0.7143
val Loss: 0.1072 Acc: 0.9815

Epoch 2/24
----------
train Loss: 0.6103 Acc: 0.7804
val Loss: 0.0470 Acc: 0.9630

Epoch 3/24
----------
train Loss: 0.6837 Acc: 0.7482
val Loss: 0.1127 Acc: 0.9630

Epoch 4/24
----------
train Loss: 0.5537 Acc: 0.8000
val Loss: 0.0077 Acc: 1.0000

Epoch 5/24
----------
train Loss: 0.5740 Acc: 0.8071
val Loss: 0.0091 Acc: 1.0000

Epoch 6/24
----------
train Loss: 0.4339 Acc: 0.8473
val Loss: 0.0033 Acc: 1.0000

Epoch 7/24
----------
train Loss: 0.3634 Acc: 0.8661
val Loss: 0.0015 Acc: 1.0000

Epoch 8/24
----------
train Loss: 0.2506 Acc: 0.9098
val Loss: 0.0013 Acc: 1.0000

Epoch 9/24
----------
train Loss: 0.3103 Acc: 0.8795
val Loss: 0.0010 Acc: 1.0000

Epoch 10/24
----------
train Loss: 0.2328 Acc: 0.9170
val Loss: 0.0015 Acc: 1.0000

Epoch 11/24
----------
train Loss: 0.2811 Acc: 0.9071
val Loss: 0.0009 Acc: 1.0000

Ep

In [ ]:
# 시각화
# R-18이기 때문에 보여주지않습니다.
visualize_model(model_ft)

In [36]:
# http://pytorch.org/docs/notes/autograd.html#excluding-subgraphs-from-backward
model_conv = torchvision.models.resnet18(pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False

# 새로 생성된 모듈의 매개변수는 기본값이 requires_grad=True 임
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 3)

model_conv = model_conv.to(device)

criterion = nn.CrossEntropyLoss()

# 이전과는 다르게 마지막 계층의 매개변수들만 최적화되는지 관찰
optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9)

# 7 에폭마다 0.1씩 학습률 감소
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

In [37]:
model_conv = train_model(model_conv, criterion, optimizer_conv,
                         exp_lr_scheduler, num_epochs=25)

Epoch 0/24
----------
train Loss: 1.0132 Acc: 0.5464
val Loss: 0.5347 Acc: 0.8148

Epoch 1/24
----------
train Loss: 0.9306 Acc: 0.6152
val Loss: 0.3583 Acc: 0.8704

Epoch 2/24
----------
train Loss: 0.7816 Acc: 0.6839
val Loss: 0.3310 Acc: 0.8889

Epoch 3/24
----------
train Loss: 0.8546 Acc: 0.6518
val Loss: 0.3121 Acc: 0.8889

Epoch 4/24
----------
train Loss: 0.8171 Acc: 0.6750
val Loss: 0.9705 Acc: 0.6111

Epoch 5/24
----------
train Loss: 0.8388 Acc: 0.6625
val Loss: 0.3213 Acc: 0.8704

Epoch 6/24
----------
train Loss: 0.7678 Acc: 0.6920
val Loss: 0.3523 Acc: 0.8704

Epoch 7/24
----------
train Loss: 0.6470 Acc: 0.7455
val Loss: 0.2595 Acc: 0.8889

Epoch 8/24
----------
train Loss: 0.6628 Acc: 0.7170
val Loss: 0.2462 Acc: 0.8889

Epoch 9/24
----------
train Loss: 0.6365 Acc: 0.7259
val Loss: 0.3084 Acc: 0.8889

Epoch 10/24
----------
train Loss: 0.6298 Acc: 0.7420
val Loss: 0.2428 Acc: 0.9074

Epoch 11/24
----------
train Loss: 0.6224 Acc: 0.7482
val Loss: 0.2634 Acc: 0.9074

Ep

총 학습시간은 1시간 30분 정도 걸렸습니다 (코랩 gpu학습 기준)

그러면 이제 단일 이미지를 사용해서 검증해봅시다.




In [51]:
# 1536576이 나와야합니다.
import PIL

with torch.no_grad():
    dtf = transforms.Compose([
        transforms.Resize(256),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
    img = PIL.Image.open('/content/1536576.png').convert('RGB')
    img = dtf(img)
    img = img.unsqueeze(0)
    # gpu로 학습된거 cpu 로바꿔줌
    model_ft.to("cpu")
    model_ft.eval()
    output = model_ft(img)
    # 가장 점수가 높은 인덱스 뽑은후 보여주기
    class_names = image_datasets['train'].classes[int(torch.max(output.data, 1)[1].numpy())]

print(output)
print(class_names)

tensor([[ 4.2470, -2.3836, -0.4999]])
1536576


In [52]:
# 1570712이 나와야합니다.
import PIL

with torch.no_grad():
    dtf = transforms.Compose([
        transforms.Resize(256),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
    img = PIL.Image.open('/content/1570712.png').convert('RGB')
    img = dtf(img)
    img = img.unsqueeze(0)
    # gpu로 학습된거 cpu 로바꿔줌
    model_ft.to("cpu")
    model_ft.eval()
    output = model_ft(img)
    # 가장 점수가 높은 인덱스 뽑은후 보여주기
    class_names = image_datasets['train'].classes[int(torch.max(output.data, 1)[1].numpy())]

print(output)
print(class_names)

tensor([[-3.3984,  2.9027, -0.0981]])
1570712


In [53]:
# 1613730이 나와야합니다.
import PIL

with torch.no_grad():
    dtf = transforms.Compose([
        transforms.Resize(256),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
    img = PIL.Image.open('/content/1613730.png').convert('RGB')
    img = dtf(img)
    img = img.unsqueeze(0)
    # gpu로 학습된거 cpu 로바꿔줌
    model_ft.to("cpu")
    model_ft.eval()
    output = model_ft(img)
    # 가장 점수가 높은 인덱스 뽑은후 보여주기
    class_names = image_datasets['train'].classes[int(torch.max(output.data, 1)[1].numpy())]

print(output)
print(class_names)

tensor([[-0.8554, -1.3595,  3.3193]])
1613730


이렇게 전이 학습을통해 망가가 성공적으로 분류될수있다는것을 확인했습니다.

일부로 같은작가의 같은 시리즈로 학습을 시켜보았는데 생각보다 인식이 잘되서 놀랐습니다.